### A. Librerias

In [ ]:
import os
import time
import json

import datetime
import pandas as pd

### B. Funciones

In [ ]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

### C. Lista de todos los investigadores de renacyt

In [ ]:
# 1. Leer los jsons y guardarlos en un dataframe
paths = get_files(os.path.join(os.getcwd(),'originals','renacyt','investigadores'), '.json')
df_renacyt = pd.concat([pd.read_json(path) for path in paths], axis=0)
df_renacyt = df_renacyt.sort_values(by=['ctiVitae'])

df_renacyt.to_parquet(os.path.join(os.getcwd(),'processed','parquet','01_lista_investigadores_renacyt.parquet'))
df_renacyt.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','01_lista_investigadores_renacyt.xlsx'), index=False)

### D. Perfiles de investtigadores de renacyt

In [ ]:
# 1. Leer los jsons y guardarlos en un dataframe
paths = get_files(os.path.join(os.getcwd(),'originals','renacyt','perfil','1_datos_investigador'), '.json')
path = paths[0]
path.split('_')[-2]

# 2. Iterar sobre cada path
formaciones_list = []
publicaciones_list = []
patentes_list= []
asesorias_list = []
for path in paths:

    # A. Obtener el orcid
    orcid = path.split('_')[-2]
    cti_vitae = path.split('_')[-3]

    # B. Leer el json
    with open(path, 'r') as archivo:
        data = json.load(archivo)

    # C. Obtener los datos de formación académica    
    formaciones = data["criterioARequest"]["formacionesAcademicas"]
    for formacion in formaciones:
        data_format = eval( '[' + str(formacion["formacionAcademicaPOJO"]) + ']') 
        df = pd.DataFrame(data_format)
        df['cti_vitae'] = cti_vitae
        df['orcid'] = orcid

        formaciones_list.append(df)

    # D. Obtener los datos de publicaciones
    publicaciones = data["criterioBRequest"]["produccionesBibliograficas"]
    for publicacion in publicaciones:
        data_format = eval( '[' + str(publicacion["produccionBibliograficaPOJO"]) + ']') 
        df = pd.DataFrame(data_format)
        df['cti_vitae'] = cti_vitae
        df['orcid'] = orcid

        publicaciones_list.append(df)

    # E. Obtener los datos de patentes
    patentes = data["criterioCRequest"]["derechosPropiedadesIntelectuales"]
    for patente in patentes:
        data_format = eval( '[' + str(patente["derechoPropiedadIntelectualPOJO"]) + ']') 
        df = pd.DataFrame(data_format)
        df['cti_vitae'] = cti_vitae
        df['orcid'] = orcid

        patentes_list.append(df)

    # F. Obtener asesorias de tesis
    asesorias = data["criterioFRequest"]["datosLaborales"]
    for asesoria in asesorias:
        data_format = eval( '[' + str(asesoria["datosLaboralesPOJO"]) + ']') 
        df = pd.DataFrame(data_format)
        df['cti_vitae'] = cti_vitae
        df['orcid'] = orcid

        asesorias_list.append(df)

In [ ]:
# 3. Concatenar formaciones_list en df_formacione y guardar 
df_formacion = pd.concat(formaciones_list, axis=0)
df_formacion.to_parquet(os.path.join(os.getcwd(),'processed','parquet','02_formaciones_academicas_renacyt.parquet'))
df_formacion = df_formacion.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_formacion.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','02_formaciones_academicas_renacyt.xlsx'), index=False)

# 4. Concatenar publicaciones_list en df_publicaciones y guardar
df_publicacion = pd.concat(publicaciones_list, axis=0)
df_publicacion.to_parquet(os.path.join(os.getcwd(),'processed','parquet','03_publicaciones_renacyt.parquet'))
df_publicacion = df_publicacion.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_publicacion.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','03_publicaciones_renacyt.xlsx'), index=False)

# 5. Concatenar patentes_list en df_patentes y guardar
df_patente = pd.concat(patentes_list, axis=0)
df_patente.to_parquet(os.path.join(os.getcwd(),'processed','parquet','04_patentes_renacyt.parquet'))
df_patente = df_patente.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_patente.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','04_patentes_renacyt.xlsx'), index=False)

# 6. Concatenar asesorias_list en df_asesorias y guardar
df_asesoria = pd.concat(asesorias_list, axis=0)
df_asesoria.to_parquet(os.path.join(os.getcwd(),'processed','parquet','05_asesorias_renacyt.parquet'))
df_asesoria = df_asesoria.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_asesoria.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','05_asesorias_renacyt.xlsx'), index=False)


### E. Perfiles de ORCID

In [ ]:
def procesar_trabajo(trabajo):
    try:
        doi = trabajo["externalIdentifiers"][0]["externalIdentifierId"]["value"]
    except:
        doi = None
    
    try:
        doi_url = trabajo["externalIdentifiers"][0]["url"]["value"]
    except:
        doi_url = None

    try:
        publication_year = trabajo["works"][0]["publicationDate"]["year"]
    except:
        publication_year = None

    try:
        publication_month = trabajo["works"][0]["publicationDate"]["month"]
    except:
        publication_month = None

    try:
        publication_day = trabajo["works"][0]["publicationDate"]["day"]
    except:
        publication_day = None

    try:
        source = trabajo["works"][0]["source"]
    except:
        source = None

    try:
        source_name = trabajo["works"][0]["sourceName"]
    except:
        source_name = None
    
    try:
        title = trabajo["works"][0]["title"]["value"]
    except:
        title = None

    try:
        journal_title = trabajo["works"][0]["journalTitle"]["value"]
    except:
        journal_title = None

    try:
        work_type = trabajo["works"][0]["workType"]["value"]
    except:
        work_type = None

    try:
        number_of_contributors = trabajo["works"][0]["numberOfContributors"]
    except:
        number_of_contributors = None

    return doi, doi_url, publication_year, publication_month, publication_day, source, source_name, title, journal_title, work_type, number_of_contributors

In [ ]:
# 1. Leer los jsons y guardarlos en un dataframe
paths = get_files(os.path.join(os.getcwd(),'originals','orcid','3_publicaciones'), '.json')
path = paths[0]

# 2. Iterar sobre cada path
trabajos_list = []
for path in paths:
    # A. Obtener el orcid
    orcid = path.split('_')[-2]
    cti_vitae = path.split('_')[-3]

    # B. Leer el json
    with open(path, 'r') as archivo:
        data = json.load(archivo)

    # C. Procesar trabajos
    trabajos = data['groups']

    print(len(trabajos))
    trabajo = trabajos[0]

    for trabajo in trabajos:
        
        doi, doi_url, publication_year, publication_month, publication_day, source, source_name, title,journal_title, work_type, number_of_contributors = procesar_trabajo(trabajo)
        
        df_trabajos = pd.DataFrame({'doi': doi,
                                    'doi_url': doi_url,
                                    'publication_year': publication_year,
                                    'publication_month': publication_month,
                                    'publication_day': publication_day,
                                    'source': source,
                                    'source_name': source_name,
                                    'title': title,
                                    'journal_title': journal_title,
                                    'work_type': work_type,
                                    'number_of_contributors': number_of_contributors}, index=[0])
        df_trabajos['cti_vitae'] = cti_vitae
        df_trabajos['orcid'] = orcid

        trabajos_list.append(df_trabajos)

# 3. Concatenar trabajos_list en df_trabajos y guardar
df_trabajo = pd.concat(trabajos_list, axis=0)
df_trabajo.to_parquet(os.path.join(os.getcwd(),'processed','parquet','06_trabajos_orcid.parquet'))
df_trabajo = df_trabajo.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_trabajo.astype(str).to_excel(os.path.join(os.getcwd(),'processed','excel','06_trabajos_orcid.xlsx'), index=False)

In [ ]:
print('Fin del procesamiento')

In [ ]:
print('Proceso finalizado')